In [58]:
import os
from pandas import read_csv, read_excel, DataFrame
import sae
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum as qsum

import sys
sys.path.append('../')
import saedfsc

Here is a notional list of suppliers.

In [59]:
saedfsc.suppliers

,Name,Location,Rating,SetUpCost
0,Supplier1,Location1,1.178742,4303
1,Supplier2,Location2,2.930649,2174
2,Supplier3,Location1,4.821298,2699
3,Supplier4,Location4,2.837146,2724
4,Supplier5,Location2,3.659921,4715
5,Supplier6,Location2,1.560158,2056
6,Supplier7,Location3,1.165378,3938


Customer information

In [60]:
saedfsc.customers

,Name,Quantity,Weights
0,CustomerType1,788,[0.73513367 0.48459612 0.93248932 0.59499169 0...
1,CustomerType2,923,[0.79095639 0.61455046 0.21374483 0.33835256 0...
2,CustomerType3,348,[0.66531545 0.17610433 0.35049571 0.43508069 0...
3,CustomerType4,295,[0.96645753 0.91481653 0.41863489 0.34691307 0...
4,CustomerType5,586,[0.06048678 0.84413304 0.82686708 0.50329165 0...
5,CustomerType6,521,[0.61118633 0.7557146 0.89788589 0.04592996 0...
6,CustomerType7,142,[0.90023275 0.01412841 0.65388829 0.09778422 0...
7,CustomerType8,313,[0.66579382 0.81465474 0.40342229 0.91312282 0...
8,CustomerType9,946,[0.07086346 0.00679177 0.92121686 0.96926981 0...
9,CustomerType10,240,[0.64659722 0.0702475 0.28557804 0.40394666 0...


Quantity discount information

In [61]:
saedfsc.qtyDiscountSchedule

,Supplier,Part,PriceLevel,MinQty,Discount
0,Supplier1,1,1,5,0.20
1,Supplier1,1,2,10,0.30
2,Supplier1,1,3,15,0.40
3,Supplier1,2,1,20,0.25
4,Supplier1,2,2,30,0.35
5,Supplier1,2,3,40,0.45
6,Supplier2,1,1,25,0.25
7,Supplier2,1,2,35,0.35
8,Supplier2,1,3,45,0.45
9,Supplier2,3,1,25,0.30


In [62]:
partOptions = saedfsc.getPartOptionsWithSuppliers()

Chad: I put in numbers for placeholders. Please put the actual numbers in.

In [63]:
nominalPartPrices = {1 : 100, 2: 300, 3 : 400}

Parameters

In [64]:
productPrice = 20000

Data structures

In [65]:
suppliers = saedfsc.suppliers['Name'].to_list()
supplierSetUpCost = dict(zip(saedfsc.suppliers['Name'], saedfsc.suppliers['SetUpCost']))
customers = saedfsc.customers['Name'].to_list()
cQty = dict(zip(saedfsc.customers['Name'], saedfsc.customers['Quantity'])) # customer quantities

unique_pairs_df = saedfsc.qtyDiscountSchedule[['Supplier', 'Part']].drop_duplicates()
suppliersPartPairs = list(unique_pairs_df.itertuples(index=False, name=None))

Chad: fill this in

In [66]:
objectives = ['mass', 'accel'] # fill this in with the objectives you want to optimize

Pricing schedule
(this code is still in progress)

In [67]:
priceLevels = {}
minQtys = {}
prices = {}
for s,p in suppliersPartPairs:
    filtered_df = saedfsc.qtyDiscountSchedule[(saedfsc.qtyDiscountSchedule['Supplier'] == s) & (saedfsc.qtyDiscountSchedule['Part'] == p)]
    priceLevels[s,p] = filtered_df['PriceLevel'].tolist()
    minQtys[s,p] = filtered_df.set_index('PriceLevel')['MinQty'].to_dict()

discounts = saedfsc.qtyDiscountSchedule.set_index(['Supplier','Part','PriceLevel'])['Discount'].to_dict()

maxQty = {}
for s,p in suppliersPartPairs:
    maxQty[s,p] = 1000

suppliersPartsAndLevels = []
for s,p in suppliersPartPairs:
    for l in priceLevels[s,p]:
        suppliersPartsAndLevels.append((s,p, l))

Create model container.

In [68]:
m = gp.Model()

Create variables

In [69]:
x = {} # part selection variables
for subsystem in partOptions.keys():
    x[subsystem] = m.addVars(partOptions[subsystem].index.values, vtype=GRB.BINARY, name="x[" + subsystem + "]")
y = m.addVars(suppliers, vtype=GRB.BINARY, name="y") # supplier selection variables
z = m.addVars(objectives, ub = 1, name="z") # objective variables (normalized to [0,1] for minimization)
v = m.addVars(suppliersPartsAndLevels, name="x") # the purchase quantity, given that the quantity of part p purchased from supplier s is in price level l
vBin = m.addVars(suppliersPartsAndLevels, vtype = GRB.BINARY, name="z") # if the quantity of part p purchased from supplier s is in price level l
w = m.addVars(customers, ub = cQty, name="w") # variables to hold customer demand

Set objective

In [70]:
m.setObjective(qsum(productPrice*cQty[c]*w[c] for c in customers) - qsum(nominalPartPrices[p]*discounts[s,p,l]*v[s,p,l] for s,p,l in suppliersPartsAndLevels) - y.prod(supplierSetUpCost), GRB.MAXIMIZE)

Quantity discount constraints

In [71]:
numLevels = {(s,p) : len(priceLevels[s,p]) for s,p in suppliersPartPairs}

suppliersPartsAndLevelsNotLast = [(s,p,l) for s,p in suppliersPartPairs for l in range(1,numLevels[s,p])]

m.addConstrs((minQtys[s,p][l]*vBin[s,p,l] <= v[s,p,l] for s,p,l in suppliersPartsAndLevels), 
                "PriceBreaks-LB")
m.addConstrs((v[s,p,l] <= minQtys[s,p][l+1]*vBin[s,p,l] for s,p,l in suppliersPartsAndLevelsNotLast), 
                "PriceBreaks-UB")
m.addConstrs((v[s,p,numLevels[s,p]] <= maxQty[s,p]*vBin[s,p,numLevels[s,p]] for s,p in suppliersPartPairs), 
                "PriceBreaks-UB-last");

Chad: add code here to set the equations for the $z$ variables (objectives)

Hugh: add code to compute customer demand based on utility